# <font style="color:rgb(50,120,229)"> Métricas de Evaluación con Keras </font>

En el cuaderno anterior, aprendiste cómo implementar métricas de evaluación para clasificación desde cero. 

Bibliotecas como Scikit-Learn ya proporcionan tales funcionalidades. De hecho, también podemos usar todas esas métricas de Keras.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd


matplotlib.style.use('ggplot')
plt.rcParams["figure.figsize"] = (10, 7)

## <font style="color:rgb(50,120,229)"> 1. Cargar los datos </font>

Vamos a cargar un conjunto de datos aleatorio  para clasificación binaria.

In [ ]:
data = pd.read_csv('./data/metrics_data.csv')

data.head()

In [ ]:
X_train = data[['X1', 'X2']].values
y_train = data['y'].values

plt.scatter(X_train[y_train == 0, 0], X_train[y_train == 0, 1], label='Class 0')
plt.scatter(X_train[y_train == 1, 0], X_train[y_train == 1, 1], label='Class 1')
plt.xlabel('X1')
plt.ylabel('X2')
plt.legend()
plt.show()

## <font style="color:rgb(50,120,229)"> 2. Definir el modelo </font>

Vamos a crear un modelo simple con una capa lineal y una función de activación sigmoide.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Input

model = Sequential()
model.add(Input(shape=(2,)))
#model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

### <font style="color:rgb(50,120,229)"> 2.1. Compilar el modelo </font>

Al compilar el modelo, podemos especificar las métricas que queremos usar.

Keras proporciona una lista de métricas que se pueden usar. Puedes encontrar más información sobre las métricas de clasificación en la [documentación oficial de Keras](https://keras.io/api/metrics/classification_metrics/).

In [ ]:
# Metricas de clasificación
from keras.metrics import  Precision, Recall, BinaryAccuracy

THRESHOLD = 0.6

precision_metric = Precision(thresholds=THRESHOLD)
recall_metric = Recall(thresholds=THRESHOLD)
accuracy_metric = BinaryAccuracy(threshold=THRESHOLD)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[precision_metric, recall_metric, BinaryAccuracy])

**Todas las métricas de clasificación estan usando los valores por defecto de `threshold=0.5`.**

La idea de estas métricas es modificar el umbral de clasificación para ver cómo se comporta el modelo.

## <font style="color:rgb(50,120,229)"> 3. Entrenar el modelo </font>

In [ ]:

model.fit(X_train, y_train, epochs=50)

## <font style="color:rgb(50,120,229)"> 4. Matriz de Confusión </font>

In [ ]:
from sklearn.metrics import confusion_matrix

THRESHOLD = 0.7

y_pred = model.predict(X_train)
y_pred = (y_pred > THRESHOLD).astype(int)

confusion_matrix(y_train, y_pred, normalize='all')

In [ ]:
import seaborn as sns

sns.heatmap(confusion_matrix(y_train, y_pred), annot=True, fmt='d', cmap='Blues')

## <font style="color:rgb(50,120,229)"> 5. Curva ROC </font>

In [ ]:
from sklearn.metrics import roc_curve, auc

THRESHOLD = 0.4

y_pred = model.predict(X_train)
y_pred = (y_pred > THRESHOLD).astype(int)

fpr, tpr, _ = roc_curve(y_train, y_pred)

roc_auc = auc(fpr, tpr)
print(roc_auc)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.legend(loc="lower right")
plt.show()